In [1]:
import pandas as pd
import numpy as np
from Data.Featurisation import data_handeler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import os
import pickle

installation_int = 0
_,_,source_data = data_handeler(installation_int, "nwp", "nwp", "nwp")
# Labels are the values we want to predict
labels = np.array(source_data['P'])
# Remove the labels from the source_data
# axis 1 refers to the columns
source_data= source_data.drop('P', axis = 1)
# Saving feature names for later use
ftr_file = "features/ft_phys.pkl"
if os.path.isfile(ftr_file):
    with open(ftr_file, 'rb') as f:
        feature_list = pickle.load(f)
# Convert to numpy array
source_data = source_data[feature_list]
print(source_data)
source_data = np.array(source_data)

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(source_data, labels, test_size = 0.25, random_state = 42)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)



Not in Colab environment: Using .pkl files


c:\Users\Robbe\SolNet-2\Data\Featurisation.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[dataset>inverter_rating] = inverter_rating


                                PoA  P_24h_shift  is_day  \
2020-05-01 00:00:00+00:00  0.000000         0.13     0.0   
2020-05-01 01:00:00+00:00  0.000000         0.13     0.0   
2020-05-01 02:00:00+00:00  0.000000         0.13     0.0   
2020-05-01 03:00:00+00:00  0.000000         0.13     0.0   
2020-05-01 04:00:00+00:00  0.020166         0.13     0.0   
...                             ...          ...     ...   
2021-04-30 19:00:00+00:00  0.000000         1.00     1.0   
2021-04-30 20:00:00+00:00  0.000000         0.00     0.0   
2021-04-30 21:00:00+00:00  0.000000         0.00     0.0   
2021-04-30 22:00:00+00:00  0.000000         0.00     0.0   
2021-04-30 23:00:00+00:00  0.000000         0.00     0.0   

                           relative_humidity_1_5m  diffuse_surface_SW_flux  \
2020-05-01 00:00:00+00:00               89.216026                      0.0   
2020-05-01 01:00:00+00:00               89.216026                      0.0   
2020-05-01 02:00:00+00:00               89.79

In [2]:
#Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels)

# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = np.sqrt(np.mean(np.square(predictions - test_labels)))
# Print out rmse
print(errors, 'W')

213.1786161618306 W


In [4]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: downward_surface_SW_flux Importance: 0.79
Variable: PoA                  Importance: 0.09
Variable: P_24h_shift          Importance: 0.03
Variable: relative_humidity_1_5m Importance: 0.02
Variable: diffuse_surface_SW_flux Importance: 0.02
Variable: direct_surface_SW_flux Importance: 0.02
Variable: T_PV                 Importance: 0.02
Variable: is_day               Importance: 0.0


[None, None, None, None, None, None, None, None]